In [9]:
import re
from spacy.tokenizer import Tokenizer
import spacy
from collections import Counter
from spacy import displacy
from spacy.matcher import Matcher
nlp = spacy.load("pl_core_news_sm")

Wstęp, dekonstrukcja stringa na pojedyncze wyrazy, oraz odczyt z pliku tekstowego

In [2]:
introduction_text = ('Przykładowy tekst do demonstracji przetwarzana Spacy')
introduction_doc = nlp(introduction_text)
print([token.text for token in introduction_doc])


['Przykładowy', 'tekst', 'do', 'demonstracji', 'przetwarzana', 'Spacy']


In [21]:
file_name = "wstep.txt"
introduction_file_text = open(file_name).read()
introduction_file_doc = nlp(introduction_file_text)
print ([token.text for token in introduction_file_doc])

['This', 'is', 'a', 'sample', 'text', '.']


Podzielenie tekstu wejśćiowego na zdania.

In [3]:
about_text=('Polityk od 1990 do 2004 sprawował mandat posła do węgierskiego Zgromadzenia Narodowego. Był m.in. przewodniczącym klubu parlamentarnego Fideszu (1994–2002) i wiceprzewodniczącym parlamentu (2002–2004). Od 1998 do 2002 kierował węgierskim komitetem integracji europejskiej. W 2000 roku wszedł w skład władz Europejskiej Partii Ludowej. W europarlamencie zasiadał nieprzerwanie od 2004 roku.')
about_doc = nlp(about_text)
sentences = list(about_doc.sents)
print(len(sentences))
for sentence in sentences:
    print(sentence)

6
Polityk od 1990 do 2004 sprawował mandat posła do węgierskiego Zgromadzenia Narodowego.
Był m.in.
przewodniczącym klubu parlamentarnego Fideszu (1994–2002) i wiceprzewodniczącym parlamentu (2002–2004).
Od 1998 do 2002 kierował węgierskim komitetem integracji europejskiej.
W 2000 roku wszedł w skład władz Europejskiej Partii Ludowej.
W europarlamencie zasiadał nieprzerwanie od 2004 roku.


Stworzenie niestandardowego znacznika końca zdania "..."

In [4]:
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == '...' or token.text == 'np.' or token.text == 'm.in.':
            doc[token.i+1].is_sent_start = True
    return doc

ellipsis_text = 'Marin, czy możesz, ... nieważne, zapomniałem  co miałem na myśli. To jak myślisz, powinniśmy ...'

custom_nlp = spacy.load("pl_core_news_sm")
custom_nlp.add_pipe(set_custom_boundaries, before='parser')
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)

Marin, czy możesz, ...
nieważne, zapomniałem  co miałem na myśli.
To jak myślisz, powinniśmy ...


Porównanie ze standardową dekompozycją

In [34]:
ellipsis_doc = nlp(ellipsis_text)
ellipsis_sentences = list(ellipsis_doc.sents)
for sentence in ellipsis_sentences:
   print(sentence)

Marin, czy możesz, ... nieważne, zapomniałem  co miałem na myśli.
To jak myślisz, powinniśmy ...


Rozbicie tekstu na tokeny

In [35]:
for token in about_doc:
    print (token, token.idx)

Polityk 0
od 8
1990 11
do 16
2004 19
sprawował 24
mandat 34
posła 41
do 47
węgierskiego 50
Zgromadzenia 63
Narodowego 76
. 86
Był 88
m.in 92
. 96
przewodniczącym 98
klubu 114
parlamentarnego 120
Fideszu 136
( 144
1994 145
– 149
2002 150
) 154
i 156
wiceprzewodniczącym 158
parlamentu 178
( 189
2002 190
– 194
2004 195
) 199
. 200
Od 202
1998 205
do 210
2002 213
kierował 218
węgierskim 227
komitetem 238
integracji 248
europejskiej 259
. 271
W 273
2000 275
roku 280
wszedł 285
w 292
skład 294
władz 300
Europejskiej 306
Partii 319
Ludowej 326
. 333
W 335
europarlamencie 337
zasiadał 353
nieprzerwanie 362
od 376
2004 379
roku 384
. 388


Wyświetlenie atrybutów tokena

In [36]:
for token in about_doc:
    print (token, token.idx, token.text_with_ws,
    token.is_alpha, token.is_punct, token.is_space,
    token.shape_, token.is_stop)

Polityk 0 Polityk  True False False Xxxxx False
od 8 od  True False False xx True
1990 11 1990  False False False dddd False
do 16 do  True False False xx True
2004 19 2004  False False False dddd False
sprawował 24 sprawował  True False False xxxx False
mandat 34 mandat  True False False xxxx False
posła 41 posła  True False False xxxx False
do 47 do  True False False xx True
węgierskiego 50 węgierskiego  True False False xxxx False
Zgromadzenia 63 Zgromadzenia  True False False Xxxxx False
Narodowego 76 Narodowego True False False Xxxxx False
. 86 .  False True False . False
Był 88 Był  True False False Xxx True
m.in 92 m.in False False False x.xx False
. 96 .  False True False . False
przewodniczącym 98 przewodniczącym  True False False xxxx False
klubu 114 klubu  True False False xxxx False
parlamentarnego 120 parlamentarnego  True False False xxxx False
Fideszu 136 Fideszu  True False False Xxxxx False
( 144 ( False True False ( False
1994 145 1994 False False False dddd False
– 1

dodanie "-" jako znacznika oddzialającego tokeny

In [39]:
custom_nlp = spacy.load('pl_core_news_sm')
prefix_re = spacy.util.compile_prefix_regex(custom_nlp.Defaults.prefixes)
suffix_re = spacy.util.compile_suffix_regex(custom_nlp.Defaults.suffixes)
infix_re = re.compile(r'''[-~]''')

def customize_tokenizer(nlp):
# Adds support to use `-` as the delimiter for tokenization
    return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                    suffix_search=suffix_re.search,
                    infix_finditer=infix_re.finditer,
                    token_match=None
                   )


custom_nlp.tokenizer = customize_tokenizer(custom_nlp)
custom_tokenizer_about_doc = custom_nlp(about_text)
print([token.text for token in custom_tokenizer_about_doc])

['Polityk', 'od', '1990', 'do', '2004', 'sprawował', 'mandat', 'posła', 'do', 'węgierskiego', 'Zgromadzenia', 'Narodowego', '.', 'Był', 'm.in', '.', 'przewodniczącym', 'klubu', 'parlamentarnego', 'Fideszu', '(', '1994–2002', ')', 'i', 'wiceprzewodniczącym', 'parlamentu', '(', '2002–2004', ')', '.', 'Od', '1998', 'do', '2002', 'kierował', 'węgierskim', 'komitetem', 'integracji', 'europejskiej', '.', 'W', '2000', 'roku', 'wszedł', 'w', 'skład', 'władz', 'Europejskiej', 'Partii', 'Ludowej', '.', 'W', 'europarlamencie', 'zasiadał', 'nieprzerwanie', 'od', '2004', 'roku', '.']


Stop words - najpopularniejsze wyrazy w danym języku, które pomijane są w analizie
gdyż nie mają znaczenia, lecz potrzebne są aby zachować sens zdania

In [40]:
import spacy
spacy_stopwords = spacy.lang.pl.stop_words.STOP_WORDS
len(spacy_stopwords)
for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)

o
dobrze
xiv
sposob
coraz
niemu
twój
nasze
mozna
dwoje


usuniecie wspomnianych wyrazow z tekstu

In [41]:
for token in about_doc:
    if not token.is_stop:
        print(token)

Polityk
1990
2004
sprawował
mandat
posła
węgierskiego
Zgromadzenia
Narodowego
.
m.in
.
przewodniczącym
klubu
parlamentarnego
Fideszu
(
1994
–
2002
)
wiceprzewodniczącym
parlamentu
(
2002
–
2004
)
.
1998
2002
kierował
węgierskim
komitetem
integracji
europejskiej
.
2000
wszedł
skład
władz
Europejskiej
Partii
Ludowej
.
europarlamencie
zasiadał
nieprzerwanie
2004
.


lista słów, które nie są stop words

In [42]:
about_no_stopword_doc = [token for token in about_doc if not token.is_stop]
print (about_no_stopword_doc)

[Polityk, 1990, 2004, sprawował, mandat, posła, węgierskiego, Zgromadzenia, Narodowego, ., m.in, ., przewodniczącym, klubu, parlamentarnego, Fideszu, (, 1994, –, 2002, ), wiceprzewodniczącym, parlamentu, (, 2002, –, 2004, ), ., 1998, 2002, kierował, węgierskim, komitetem, integracji, europejskiej, ., 2000, wszedł, skład, władz, Europejskiej, Partii, Ludowej, ., europarlamencie, zasiadał, nieprzerwanie, 2004, .]


Lemmanization - usunięcie odmiany wyrazów, zachowując jednocześnie ich sens

In [44]:
conference_help_text = ('Paweł pomaga zoragnizować programistyczną '
'konferencje o zastosowaniach języka'
' i jego przetwarzaniu. Organizuje spotkania'
' i wiele wewnętrznych osób rozmawia w jego imieniu')
conference_help_doc = nlp(conference_help_text)
for token in conference_help_doc:
    print (token, token.lemma_)

Paweł Pawła
pomaga pomagać
zoragnizować zoragnizować
programistyczną programistyczny
konferencje konferencja
o o
zastosowaniach zastosowanie
języka język
i i
jego on
przetwarzaniu przetwarzanie
. .
Organizuje organizować
spotkania spotkanie
i i
wiele wiele
wewnętrznych wewnętrzny
osób osoba
rozmawia rozmawiać
w w
jego on
imieniu imię


Częstotliwość występywania słów

In [6]:
complete_text = 'Polska pozostaje pod wpływem potężnego, zimnego rosyjskiego wyżu, w masie zimnego starego powietrza arktycznego. Będzie ona od czwartku odsuwana znad Polski przez cieplejsze masy polarne, pchane do nas przez pochmurne wiry niżowe znad Atlantyku, zachodniej Europy i Morza Śródziemnego.Temperatura w drugiej połowie tygodnia wzrośnie, podobnie jak zachmurzenie, a mróz zelżeje. Nie ma co liczyć na zimową aurę w mikołajkowy weekend. Zapowiada się plucha.' \
                'W środę będzie jeszcze zimno, ale pogodnie. Miejscami przejściowo wzrośnie zachmurzenie. Temperatura minimalna wyniesie od -1 stopnia Celsjusza na Suwalszczyźnie, Podlasiu oraz w Małopolsce i na Górnym Śląsku, przez  1 st. C w centrum kraju, do 3 st. C na Ziemi Lubuskiej i Pomorzu Zachodnim. Wiatr będzie wschodni i południowo-wschodni, słaby i umiarkowany, okresami silniejszy.'
complete_doc = nlp(complete_text)
# usuwamy słowa stop i znaki interpunkcyjne
words = [token.text for token in complete_doc
        if not token.is_stop and not token.is_punct]
word_freq = Counter(words)

5 najczęściej występujących wyrazów wraz z liczbą wystąpień

In [50]:
common_words = word_freq.most_common(5)
print (common_words)

[('zimnego', 2), ('znad', 2), ('Temperatura', 2), ('wzrośnie', 2), ('zachmurzenie', 2)]


unikatowe wyrazy (występujące tylko raz)

In [51]:
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

['Polska', 'pozostaje', 'wpływem', 'potężnego', 'rosyjskiego', 'wyżu', 'masie', 'starego', 'powietrza', 'arktycznego', 'czwartku', 'odsuwana', 'Polski', 'cieplejsze', 'masy', 'polarne', 'pchane', 'pochmurne', 'wiry', 'niżowe', 'Atlantyku', 'zachodniej', 'Europy', 'Morza', 'Śródziemnego', 'drugiej', 'połowie', 'tygodnia', 'podobnie', 'mróz', 'zelżeje', 'liczyć', 'zimową', 'aurę', 'mikołajkowy', 'weekend', 'Zapowiada', 'plucha', 'środę', 'zimno', 'pogodnie', 'Miejscami', 'przejściowo', 'minimalna', 'wyniesie', '-1', 'stopnia', 'Celsjusza', 'Suwalszczyźnie', 'Podlasiu', 'Małopolsce', 'Górnym', 'Śląsku', ' ', '1', 'centrum', 'kraju', '3', 'Ziemi', 'Lubuskiej', 'Pomorzu', 'Zachodnim', 'Wiatr', 'południowo', 'słaby', 'umiarkowany', 'okresami', 'silniejszy']


Wyodrębnianie i rozpoznawanie części mowy

In [52]:
for token in about_doc:
    print (token, token.tag_, token.pos_, spacy.explain(token.tag_))

Polityk SUBST NOUN None
od PREP ADP None
1990 ADJ ADJ adjective
do PREP ADP None
2004 ADJ ADJ adjective
sprawował PRAET VERB None
mandat SUBST NOUN None
posła SUBST NOUN None
do PREP ADP None
węgierskiego ADJ ADJ adjective
Zgromadzenia SUBST NOUN None
Narodowego ADJ ADJ adjective
. INTERP PUNCT None
Był PRAET VERB None
m.in BREV X None
. INTERP PUNCT None
przewodniczącym SUBST NOUN None
klubu SUBST NOUN None
parlamentarnego ADJ ADJ adjective
Fideszu SUBST NOUN None
( INTERP PUNCT None
1994 NUM NUM numeral
– INTERP PUNCT None
2002 NUM NUM numeral
) INTERP PUNCT None
i CONJ CCONJ conjunction
wiceprzewodniczącym SUBST NOUN None
parlamentu SUBST NOUN None
( INTERP PUNCT None
2002 NUM NUM numeral
– INTERP PUNCT None
2004 NUM NUM numeral
) INTERP PUNCT None
. INTERP PUNCT None
Od PREP ADP None
1998 ADJ ADJ adjective
do PREP ADP None
2002 ADJ ADJ adjective
kierował PRAET VERB None
węgierskim ADJ ADJ adjective
komitetem SUBST NOUN None
integracji SUBST NOUN None
europejskiej ADJ ADJ adjective


Wyodrębnienie tylko wskazanych częśći mowy

In [56]:
nouns = []
adjectives = []
for token in about_doc:
    if token.pos_ == 'NOUN':
        nouns.append(token)
    if token.pos_ == 'ADJ':
        adjectives.append(token)
print('nouns:',nouns, '\n');
print('adjectives:',adjectives, '\n');

nouns: [Polityk, mandat, posła, Zgromadzenia, przewodniczącym, klubu, Fideszu, wiceprzewodniczącym, parlamentu, komitetem, integracji, roku, skład, władz, Partii, europarlamencie, roku] 

adjectives: [1990, 2004, węgierskiego, Narodowego, parlamentarnego, 1998, 2002, węgierskim, europejskiej, 2000, Europejskiej, Ludowej, 2004] 



wizualizacja

In [8]:
about_interest_text = ('Krowy dają mleko, ale nie napoje gazowane')
about_interest_doc = nlp(about_interest_text)
displacy.render(about_interest_doc, style='dep', jupyter=True)

# Funkcje przygotowujące dane

funkcja zamieniająca tekst, na dane gotowe do przetowrzenia za pomocą spacy

In [7]:
def is_token_allowed(token):
    if (not token or not token.string.strip() or
         token.is_stop or token.is_punct):
         return False
    return True

def preprocess_token(token):
     return token.lemma_.strip().lower()

complete_filtered_tokens = [preprocess_token(token)
    for token in complete_doc if is_token_allowed(token)]
complete_filtered_tokens

['polska',
 'pozostawać',
 'wpływ',
 'potężny',
 'zimny',
 'rosyjski',
 'wyż',
 'masa',
 'zimny',
 'stary',
 'powietrze',
 'arktyczny',
 'czwartek',
 'odsuwać',
 'znad',
 'polska',
 'ciepły',
 'masa',
 'polarny',
 'pchać',
 'pochmurny',
 'wir',
 'niżowy',
 'znad',
 'atlantyk',
 'zachodni',
 'europ',
 'morze',
 'śródziemny',
 'temperatura',
 'drugi',
 'połów',
 'tydzień',
 'wzrosnąć',
 'podobnie',
 'zachmurzenie',
 'mróz',
 'zelżeje',
 'liczyć',
 'zimowy',
 'aura',
 'mikołajkowy',
 'weekend',
 'zapowiadać',
 'plucha',
 'środa',
 'zimno',
 'pogodnie',
 'miejsce',
 'przejściowo',
 'wzrosnąć',
 'zachmurzenie',
 'temperatura',
 'minimalny',
 'wynieść',
 '-1',
 'stopień',
 'celsjusz',
 'suwalszczyzna',
 'podlasie',
 'małopolska',
 'górny',
 'śląsko',
 '1',
 'st',
 'c',
 'centrum',
 'kraj',
 '3',
 'st',
 'c',
 'ziemia',
 'lubuski',
 'pomorze',
 'zachodni',
 'wiatr',
 'wschodni',
 'południowy',
 'wschodni',
 'słaby',
 'umiarkowany',
 'okres',
 'silny']

Wyodrębnienie imienia i nazwiska

In [13]:
matcher = Matcher(nlp.vocab)
tekst_z_nazwiskiem = 'Marcin Kowalski jest mistrzem świata w zalewaniu herbaty zimną wodą.'
nlp_nazwisko = nlp(tekst_z_nazwiskiem)
def extract_full_name(nlp_doc):
     pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
     matcher.add('FULL_NAME', None, pattern)
     matches = matcher(nlp_doc)
     for match_id, start, end in matches:
         span = nlp_nazwisko[start:end]
         return span.text

extract_full_name(nlp_nazwisko)

Wyodrębnienie numeru telefonu

In [14]:
conference_org_text = ('Możemy umówićsięw innym terminie, tu jest mój numer telefonu: (123) 456-789')

def extract_phone_number(nlp_doc):
     pattern = [{'ORTH': '('}, {'SHAPE': 'ddd'},
                {'ORTH': ')'}, {'SHAPE': 'ddd'},
                {'ORTH': '-', 'OP': '?'},
                {'SHAPE': 'ddd'}]
     matcher.add('PHONE_NUMBER', None, pattern)
     matches = matcher(nlp_doc)
     for match_id, start, end in matches:
         span = nlp_doc[start:end]
         return span.text

conference_org_doc = nlp(conference_org_text)
extract_phone_number(conference_org_doc)

'(123) 456-789'

# zależności między wyrazami

 wyświetlenie zależności

In [37]:
matma_text = 'Gus maciek uczy się matematyki'
matma_doc = nlp(matma_text)
for token in matma_doc:
    print (token.text, token.tag_, token.head.text, token.dep_)
displacy.render(matma_doc, style='dep', jupyter=True)

Gus SUBST uczy nsubj
maciek SUBST Gus nmod
uczy FIN uczy ROOT
się QUB uczy expl:pv
matematyki SUBST uczy obj


In [ ]:
one_line_about_text = ('Marcin Niekłoński jest starszym programistą w firmie Microsoft')
one_line_about_doc = nlp(one_line_about_text)

dzieci od 'programista'

In [23]:
print([token.text for token in one_line_about_doc[4].children])

['Marcin', 'jest', 'starszym', 'firmie']


Poprzedni węzeł od `Microsoftu`

In [25]:
print (one_line_about_doc[6].nbor(-1))

w


Następny węzeł od 'jest'

In [30]:
print (one_line_about_doc[3].nbor())

programistą


Wyodrębnij wszystkie węzły na lewo od `programista`

In [28]:
print([token.text for token in one_line_about_doc[4].lefts])

['Marcin', 'jest', 'starszym']


Na prawo od `programista'

In [34]:
print([token.text for token in one_line_about_doc[4].rights])

['firmie']


wyświetl poddrzewo `programista`

In [35]:
print (list(one_line_about_doc[4].subtree))

[Marcin, Niekłoński, jest, starszym, programistą, w, firmie, Microsoft]


wyciągnie kontekstu zdania

In [38]:
gig_text = ('Dnia 23-03 odbędzie się koncert grupy Bayer Full w Filharmonii warszawskiej')
conference_doc = nlp(gig_text)
# Extract Noun Phrases
for chunk in conference_doc.noun_chunks:
     print (chunk)

Wyodrębnienie na postawie gramatyki (pattern)

In [42]:
import textacy
about_talk_text = ('Rozmowa wprowadzi w użytkownika w niesmowity świat tajemniczej magii i czarów')
pattern = r'(<VERB>?<ADV>*<VERB>+)'
about_talk_doc = textacy.make_spacy_doc(about_talk_text,
                                        lang='pl_core_news_sm')
verb_phrases = textacy.extract.pos_regex_matches(about_talk_doc, pattern)
# Print all Verb Phrase
for chunk in verb_phrases:
     print(chunk.text)

wprowadzi


c:\users\dzem0\pycharmprojects\pythonproject\venv\lib\site-packages\textacy\extract.py:334: DeprecationWarning: `pos_regex_matches()` has been deprecated! for similar but more powerful and performant functionality, use `textacy.extract.matches()` instead.
  utils.deprecated(


Rozpoznawanie nazw własnych

In [47]:
piano_class_text = ('Wielki Kanion leży w Stanach Zjednoczonych, a Wielki Mur Chiński - w Chinach')
piano_class_doc = nlp(piano_class_text)
for ent in piano_class_doc.ents:
     print(ent.text, ent.start_char, ent.end_char,
           ent.label_, spacy.explain(ent.label_))
displacy.render(piano_class_doc, style='ent', jupyter=True)

Kanion 7 13 placeName None
Stanach Zjednoczonych 21 42 placeName None
Wielki 46 52 placeName None
Chiński 57 64 placeName None
Chinach 69 76 placeName None


usuwanie imion

In [49]:
survey_text = 'Okazuje się, że Marcin Nowak lubi owoce, Grażyna Kowal bardziej wazrywa, a Wiesław Miecz je tylko mięso'

def replace_person_names(token):
     if token.ent_iob != 0 and token.ent_type_ == 'PERSON':
         return '[REDACTED] '
     return token.string

def redact_names(nlp_doc):
     for ent in nlp_doc.ents:
         ent.merge()
     tokens = map(replace_person_names, nlp_doc)
     return ''.join(tokens)

survey_doc = nlp(survey_text)
redact_names(survey_doc)


<ipython-input-49-89f81c1a5793>:10: DeprecationWarning: [W013] As of v2.1.0, Span.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.
  ent.merge()
<ipython-input-49-89f81c1a5793>:10: DeprecationWarning: [W013] As of v2.1.0, Doc.merge is deprecated. Please use the more efficient and less error-prone Doc.retokenize context manager instead.
  ent.merge()


'Okazuje się, że Marcin Nowak lubi owoce, Grażyna Kowal bardziej wazrywa, a Wiesław Miecz je tylko mięso'